In [22]:
import time
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import matplotlib.pyplot as plt


import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

#Method that gets Bitcoin price from KRAKEN exchange 
def get_quandl_data(quandl_id):
    assert isinstance(quandl_id, str), "quandl_id must be a sring"
    assert quandl_id == 'BCHARTS/KRAKENUSD', "quandl_id must be from Kraken to work properly"
    '''Download and cache Quandl dataseries '''
    cache_path = '{}.pkl'.format(quandl_id).replace('/', '-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas", authtoken='kJe2AzksfSUCSuhEpMBt')
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

#store bitcoin price information in here
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

#lets analyze the price of bitcoin for the three months
btc_usd_price_kraken = btc_usd_price_kraken['2017-11-01':'2017-11-30']

#let us check if we have some good info at the top
btc_usd_price_kraken.head()





Loaded BCHARTS/KRAKENUSD from cache


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2017-11-01,6442.3,6749.0,6362.0,6741.9,5055.978944,3.309668e+07,6546.048195
2017-11-02,6741.8,7289.0,6710.8,7007.3,7346.374226,5.118178e+07,6966.943635
2017-11-03,7020.3,7449.0,6894.1,7160.0,4840.042536,3.507105e+07,7246.020896
2017-11-04,7160.0,7480.0,6934.1,7369.9,2705.035039,1.958950e+07,7241.866100
2017-11-05,7366.5,7575.0,7297.0,7380.0,2604.588509,1.940652e+07,7450.897130


In [23]:
#same thing with the end of the table
btc_usd_price_kraken.tail()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2017-11-26,8691.8,9388.0,8691.8,9174.3,3192.032080,2.880184e+07,9023.042380
2017-11-27,9174.3,9654.7,9155.6,9580.0,3131.805338,2.973401e+07,9494.206005
2017-11-28,9597.7,9885.2,9411.0,9800.0,2475.904112,2.408367e+07,9727.223970
2017-11-29,9820.0,11379.8,9000.2,9795.8,9958.758730,1.023290e+08,10275.273668
2017-11-30,9773.1,10680.1,9220.8,10088.5,6997.111208,6.923076e+07,9894.191618


In [24]:

#get url for Poloniex Exchange for altcoin prices
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2017-11-01', '%Y-%m-%d') # get data from the start of November 2017
end_date = datetime.strptime('2017-11-30', '%Y-%m-%d') # to the end of November 2017
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    assert isinstance(json_url, str), "Please make sure it is string"
    assert isinstance(cache_path, str), "Please make sure is string"
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached response at {}'.format(json_url, cache_path))
    return df

def get_crypto_data(poloniex_pair):
    assert (poloniex_pair == 'BTC_ETH' or poloniex_pair == 'BTC_LTC' 
            or poloniex_pair == 'BTC_ZEC' or poloniex_pair == 'BTC_DASH'), "Please input any of the following pairs:" + " BTC_ETH or BTC_LTC or BTC_ZEC or BTC_DASH"
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, time.mktime(start_date.timetuple()), time.mktime(end_date.timetuple()), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

#import some of the top 20 cryptocurrencies
altcoins = ['ETH', 'LTC', 'ZEC' ]
altcoin_data = {}

#in this loop get all altcoin info from cache
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

#add dash a top 5 crypto to the list to compare
privacycoins = ['DASH', 'LTC', 'ETH', 'ZEC']
privacy_data = {}

#in this loop get all altcoin and DASH info from cache
for coin in privacycoins:
    coinpair = 'BTC_{}'.format(coin)
    lol = get_crypto_data(coinpair)
    privacy_data[coin] = lol
    
    

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ZEC&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1509519600.0&end=1512028800.0&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ZEC&start=1509519600.0&end=1512028800.0&period=86400 from cache


In [25]:
#calculate USD price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] = altcoin_data[altcoin]['weightedAverage']*btc_usd_price_kraken['Weighted Price']   

def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)    
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig) 
    
# Merge USD price of each altcoin into single dataframe
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')
combined_df['BTC'] = btc_usd_price_kraken['Weighted Price']

#make plot of BTC, ETH, ZEC, LTC
df_scatter(combined_df['2017-11-01':'2017-11-30'], 'Values of Cryptocurrencies', seperate_y_axis=False, y_axis_label='Value of Coin (USD)', scale ='log')

#calculate USD price as a new column in each altcoin and DASH dataframe
for coin in privacy_data.keys():
    privacy_data[coin]['price_usd'] = privacy_data[coin]['weightedAverage']*btc_usd_price_kraken['Weighted Price']

# Merge USD price of each altcoin and DASH into single dataframe
privacy_df = merge_dfs_on_column(list(privacy_data.values()), list(privacy_data.keys()), 'price_usd')
privacy_df['BTC'] = btc_usd_price_kraken['Weighted Price']

#produce the plot of DASH, BTC, ETH, ZEC, LTC
df_scatter(privacy_df['2017-11-01':'2017-11-30'], 'Values of Cryptocurrencies', seperate_y_axis=False, y_axis_label='Value of Coin (USD)', scale='log')


In [26]:
#find the return of each day in November 2017
crypto_returns = combined_df["2017-11-01":"2017-11-30"].apply(lambda x: x/x[0])
#find the return of each day in Novemer 2017 - DASH included
crypto_privacy_returns = privacy_df["2017-11-01":"2017-11-30"].apply(lambda x:x/x[0])


In [28]:
#plot the growth rate of each of the currencies
#crypto_returns.plot(grid = True).axhline(y=1, color="black", lw=2)
crypto_privacy_returns.plot(grid = True).axhline(y=1, color="black", lw=2)


In [16]:
#calculate the pearson correlation coefficients for BTC, ETH, ZEC, LTC in November 2017
combined_df_2017 = combined_df['2017-11-01':'2017-11-30']
combined_df_2017.pct_change().corr(method='pearson')


,ETH,LTC,ZEC,BTC
ETH,1.000000,0.618937,0.489348,0.280574
LTC,0.618937,1.000000,0.595762,0.610553
ZEC,0.489348,0.595762,1.000000,0.340072
BTC,0.280574,0.610553,0.340072,1.000000


In [17]:
#calculate the pearson correlation coefficients for BTC, DOGE, DASH, XMR in November 2017
privacy_df_2017 = privacy_df['2017-11-01':'2017-11-30']
privacy_df_2017.pct_change().corr(method='pearson')

,DASH,ETH,LTC,ZEC,BTC
DASH,1.000000,-0.131905,-0.181615,0.045686,-0.313428
ETH,-0.131905,1.000000,0.618937,0.489348,0.280574
LTC,-0.181615,0.618937,1.000000,0.595762,0.610553
ZEC,0.045686,0.489348,0.595762,1.000000,0.340072
BTC,-0.313428,0.280574,0.610553,0.340072,1.000000


In [18]:
#create the correlation heatmap of growth rates of cryptocurrencies
def correlation_heatmap(df, title, absolute_bounds=True):
    assert isinstance(title, str), "title must be a string"
    assert isinstance(df, pd.DataFrame), "df must be pandas.core.frame.DataFrame"
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)
    
#get peason correlation of BTC,LTC,ETH,DASH    
correlation_heatmap(combined_df_2017.pct_change(), "Top Cryptocurrencies' Correlations in November 2017")
correlation_heatmap(privacy_df_2017.pct_change(), "Private Cryptocurrencies' Correlations in November 2017")